# Introduction

In this notebook, we will develop a prototype pricing model for a natural gas trading contract. The goal is to determine the value of the contract based on various input parameters such as injection dates, withdrawal dates, prices, injection/withdrawal rates, maximum storage volume, and storage costs.

# Background

The client believes that the upcoming winter will be colder than expected and wants to buy natural gas now to store and sell later, taking advantage of the anticipated increase in gas prices. The pricing model will help the trading desk evaluate the profitability of the contract by considering all associated costs and revenues.

# Objective

The objective of this notebook is to create a function that calculates the value of the trading contract. The function will take into account the following input parameters:

*   Injection dates
*   Withdrawal dates
*   Prices at which the commodity can be purchased/sold on those dates
*   Rate at which the gas can be injected/withdrawn
*   Maximum volume that can be stored
*   Storage costs

In [5]:
# import the required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv('Nat_Gas.csv')
df.head(5)

,Dates,Prices
0,10/31/20,10.1
1,11/30/20,10.3
2,12/31/20,11.0
3,1/31/21,10.9
4,2/28/21,10.9


# Function Documentation

### `price_contract`

The `price_contract` function calculates the value of a natural gas trading contract based on various input parameters. The function considers the costs associated with injecting, storing, and withdrawing natural gas, as well as the revenues generated from selling the gas.

#### Parameters

- **`injection_dates` (list of str)**: List of dates when natural gas is injected into the storage facility. The dates should be in the format 'YYYY-MM-DD'.
- **`withdrawal_dates` (list of str)**: List of dates when natural gas is withdrawn from the storage facility. The dates should be in the format 'YYYY-MM-DD'.
- **`prices` (dict)**: Dictionary where the keys are dates (in the format 'YYYY-MM-DD') and the values are the prices of natural gas on those dates.
- **`injection_rate` (float)**: The rate at which natural gas can be injected into the storage facility, in MMBtu.
- **`withdrawal_rate` (float)**: The rate at which natural gas can be withdrawn from the storage facility, in MMBtu.
- **`max_volume` (float)**: The maximum volume of natural gas that can be stored in the facility, in MMBtu.
- **`storage_cost_per_month` (float)**: The monthly cost of storing natural gas, in dollars.
- **`injection_withdrawal_cost_per_mmbtu` (float)**: The cost per MMBtu for injecting or withdrawing natural gas, in dollars.
- **`transport_cost` (float)**: The cost of transporting natural gas to and from the storage facility, in dollars.

#### Returns

- **`contract_value` (float)**: The calculated value of the natural gas trading contract, in dollars.

#### Steps

1. **Date Conversion**: Convert the injection and withdrawal dates to datetime objects for easier manipulation.
2. **Price DataFrame**: Create a DataFrame from the prices dictionary to facilitate lookup of prices by date.
3. **Initialize Variables**: Initialize variables to keep track of total revenue, total cost, and current volume of natural gas in storage.
4. **Calculate Injection Costs**:
   - For each injection date, retrieve the price of natural gas.
   - Calculate the volume of natural gas to be injected, ensuring it does not exceed the maximum storage volume.
   - Update the current volume and calculate the injection cost, including the purchase cost and injection fee.
   - Add the injection cost to the total cost.
5. **Calculate Storage Costs**:
   - Determine the number of months the natural gas will be stored.
   - Calculate the total storage cost based on the monthly storage cost.
   - Add the storage cost to the total cost.
6. **Calculate Withdrawal Revenues**:
   - For each withdrawal date, retrieve the price of natural gas.
   - Calculate the volume of natural gas to be withdrawn, ensuring it does not exceed the current volume in storage.
   - Update the current volume and calculate the withdrawal revenue, including the sale revenue and withdrawal fee.
   - Add the withdrawal revenue to the total revenue.
7. **Calculate Transport Costs**:
   - Calculate the total transport cost, assuming it is incurred twice (to and from the facility).
   - Add the transport cost to the total cost.
8. **Calculate Contract Value**:
   - Subtract the total cost from the total revenue to determine the value of the contract.
   - Return the contract value.

In [11]:
def price_contract(injection_dates,
                   withdrawal_dates,
                   prices,
                   injection_rate,
                   withdrawal_rate,
                   max_volume,
                   storage_cost_per_month,
                   injection_withdrawal_cost_per_mmbtu,
                   transport_cost):

    # Convert the injection and withdrawal dates to datetime objects
    injection_dates = pd.to_datetime(injection_dates)
    withdrawal_dates = pd.to_datetime(withdrawal_dates)

    # Create a DataFrame to facilitate lookup of prices by date
    price_data = pd.DataFrame({'Dates': prices.keys(),
                               'Prices': prices.values()})
    price_data['Dates'] = pd.to_datetime(price_data['Dates'])

    # Initialize variables
    total_revenue = 0
    total_cost = 0
    current_volume = 0

    # Calculate injection costs and volumes
    print("Injection Costs:")

    for date in injection_dates:

        # For each injection date, retrieve the price of natural gas
        price = price_data[price_data['Dates'] == date]['Prices'].values[0]

        # Calculate the volume of natural gas to be injected
        volume = min(injection_rate, max_volume - current_volume)

        # Update the current volume
        current_volume += volume

        #  Calculate the injection cost
        injection_cost = (price * volume) + (injection_withdrawal_cost_per_mmbtu * volume / 1e6)

        # Add the injection cost to the total cost
        total_cost += injection_cost

        print(f"Date: {date} \nPrice: ${price}, Volume: {volume} MMBtu, Injection Cost: ${injection_cost:.2f} \n")

    # Calculate storage costs
    storage_months = (withdrawal_dates.max() - injection_dates.min()).days // 30
    storage_cost = storage_cost_per_month * storage_months
    total_cost += storage_cost
    print(f"Storage Costs: ${storage_cost:.2f} for {storage_months} months \n")

    # Calculate withdrawal revenues and volumes
    print("Withdrawal Revenues:")
    for date in withdrawal_dates:

        # Retrieve the price of natural gas
        price = price_data[price_data['Dates'] == date]['Prices'].values[0]

        # Calculate the volume of natural gas to be withdrawn
        volume = min(withdrawal_rate, current_volume)

        # Update the current volume
        current_volume -= volume

        # Calculate the withdrawal revenue
        withdrawal_revenue = (price * volume) - (injection_withdrawal_cost_per_mmbtu * volume / 1e6)

        # Add the withdrawal revenue to the total revenue
        total_revenue += withdrawal_revenue

        print(f"Date: {date} \nPrice: ${price}, Volume: {volume} MMBtu, Withdrawal Revenue: ${withdrawal_revenue:.2f} \n")

    # Calculate the total transport cost assuming it is incurred twice (to and from the facility)
    transport_costs = transport_cost * 2
    total_cost += transport_costs
    print(f"Transport Costs: ${transport_costs:.2f} \n")

    # Calculate the value of the contract
    contract_value = total_revenue - total_cost
    print(f"Total Revenue: ${total_revenue:.2f} \n")
    print(f"Total Cost: ${total_cost:.2f} \n")
    print(f"Contract Value: ${contract_value:.2f} \n")

    return contract_value

In [12]:
# inputs
injection_dates = ['2020-10-31', '2021-02-28']
withdrawal_dates = ['2024-09-30', '2024-02-29']
prices = {
    '2020-10-31': 10.1,
    '2021-02-28': 10.9,
    '2024-09-30': 11.8,
    '2024-02-29': 12.4
}
injection_rate = 1e6  # MMBtu
withdrawal_rate = 1e6  # MMBtu
max_volume = 2e6  # MMBtu
storage_cost_per_month = 100e3  # $100K per month
injection_withdrawal_cost_per_mmbtu = 10e3  # $10K per 1 million MMBtu
transport_cost = 50e3  # $50K per transport

# Final output
contract_value = price_contract(injection_dates,
                                withdrawal_dates,
                                prices,
                                injection_rate,
                                withdrawal_rate,
                                max_volume,
                                storage_cost_per_month,
                                injection_withdrawal_cost_per_mmbtu,
                                transport_cost)

print(f"The value of the contract is: ${contract_value:.2f}")

Injection Costs:
Date: 2020-10-31 00:00:00 
Price: $10.1, Volume: 1000000.0 MMBtu, Injection Cost: $10110000.00 

Date: 2021-02-28 00:00:00 
Price: $10.9, Volume: 1000000.0 MMBtu, Injection Cost: $10910000.00 

Storage Costs: $4700000.00 for 47 months 

Withdrawal Revenues:
Date: 2024-09-30 00:00:00 
Price: $11.8, Volume: 1000000.0 MMBtu, Withdrawal Revenue: $11790000.00 

Date: 2024-02-29 00:00:00 
Price: $12.4, Volume: 1000000.0 MMBtu, Withdrawal Revenue: $12390000.00 

Transport Costs: $100000.00 

Total Revenue: $24180000.00 

Total Cost: $25820000.00 

Contract Value: $-1640000.00 

The value of the contract is: $-1640000.00


 The negative contract value indicates that the costs associated with the storage and trading of natural gas exceed the revenues generated from selling the gas.

# Calculation breakdown

Let's break down the calculations step by step to ensure everything is correct:

**Injection Costs:**

*2020-10-31:*

Price: $10.1 per MMBtu

Volume: 1,000,000 MMBtu

Injection Cost: $10,100,000 (purchase cost) + $10,000 (injection fee) = $10,110,000

*2021-02-28:*

Price: $10.9 per MMBtu

Volume: 1,000,000 MMBtu

Injection Cost: $10,900,000 (purchase cost) + $10,000 (injection fee) = $10,910,000

Total injection costs = $10,110,000 + $10,910,000 = $21,020,000

**Storage Costs:**

Storage period: 47 months

Storage cost: $100,000 per month

Total storage cost: 47 * $100,000 = $4,700,000

**Withdrawal Revenues:**

*2024-09-30:*

Price: $11.8 per MMBtu

Volume: 1,000,000 MMBtu

Withdrawal Revenue: $11,800,000 (sale revenue) - $10,000 (withdrawal fee) = $11,790,000

*2024-02-29:*

Price: $12.4 per MMBtu

Volume: 1,000,000 MMBtu

Withdrawal Revenue: $12,400,000 (sale revenue) - $10,000 (withdrawal fee) = $12,390,000


Total withdrawal revenues = $11,790,000 + $12,390,000 = $24,180,000

**Transport Costs:**

Transport cost: $50,000 per transport

Total transport cost: 2 * $50,000 = $100,000

Total Costs:

Injection costs: $21,020,000

Storage costs: $4,700,000

Transport costs: $100,000

Total costs = $21,020,000 + $4,700,000 + $100,000 = $25,820,000

**Total Revenues:**

Withdrawal revenues: $24,180,000

**Contract Value:**

Contract value = Total revenues - Total costs

Contract value = $24,180,000 - $25,820,000 = -$1,640,000

The calculations confirm that the contract value is indeed negative, indicating a loss. The detailed breakdown shows that the costs (injection, storage, and transport) exceed the revenues from selling the gas.

# Conclusion

The contract is not profitable under the given parameters. To make the contract profitable, you may need to:

*   **Reduce Injection and Withdrawal Costs:** Negotiate lower fees for injection and withdrawal.
*   **Optimize Storage Costs:** Find a storage facility with lower monthly costs.
*   **Increase Selling Prices:** Aim to sell the gas at higher prices during withdrawal.
*   **Reduce Transport Costs:** Negotiate lower transport costs or find a more cost-effective transport method.

If you need to adjust any of these parameters to see how they affect the contract value, you can modify the input values and re-run the function.

# Disclaimer

The calculations and conclusions are based on the sample data provided. The actual contract value may vary depending on real-world market conditions, accurate cost estimates, and other factors not accounted for in this simplified model. It is recommended to validate the model with real data and consider additional factors before making any trading decisions.